<a href="https://colab.research.google.com/github/cpython-projects/da_1709/blob/main/lesson_25_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Підзапити та CTE в SQL


## Підзапит (Subquery)

Підзапит — це **запит всередині іншого запиту**. Він може використовуватися в:

* `SELECT`:
  → щоб повернути одне значення (скаляр)
* `FROM`:
  → як тимчасова таблиця
* `WHERE` / `IN` / `EXISTS`:
  → для фільтрації


### Приклад підзапиту в `SELECT`


In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import plotly.express as px

In [2]:
DB_USER = "prog_academy_da_yuq6_user"
DB_PASS = "nKLaf3orGpgat1GEpFbkuNVtxcNAGJp1"
DB_HOST = "dpg-d519e97pm1nc73briqk0-a.oregon-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "prog_academy_da_yuq6"

In [3]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [4]:
# Рахуємо загальну кількість замовлень кожного користувача та окремо — його середній чек:

query = """
SELECT
    user_uuid,
    COUNT(*) AS order_count,
    AVG(total_uah) AS avg_check1,
    (
      SELECT AVG(total_uah)
     FROM orders_log o2
     WHERE o2.user_uuid = o1.user_uuid) AS avg_check
FROM orders_log o1
GROUP BY user_uuid;
"""

df = pd.read_sql(text(query), engine)
df.head()

,user_uuid,order_count,avg_check1,avg_check
0,USR0107,5,989.436000,989.436000
1,USR0066,13,1466.497692,1466.497692
2,USR0106,14,1369.988571,1369.988571
3,USR0224,9,2032.513333,2032.513333
4,USR0138,14,1351.451429,1351.451429


In [7]:
# Рахуємо загальну кількість замовлень кожного користувача та окремо — його середній чек:

query = """
SELECT
    user_uuid,

    (
      SELECT AVG(total_uah)
     FROM orders_log o2
     WHERE o2.user_uuid = o1.user_uuid) AS avg_check
FROM orders_log o1;
"""

df = pd.read_sql(text(query), engine)
df

,user_uuid,avg_check
0,USR0060,2211.610
1,USR0060,2211.610
2,USR0060,2211.610
3,USR0060,2211.610
4,USR0060,2211.610
...,...,...
2554,USR0211,1494.484
2555,USR0211,1494.484
2556,USR0211,1494.484
2557,USR0211,1494.484


### Приклад підзапиту в `FROM`:


In [ ]:
# Обчислимо дохід по кожному користувачу в підзапиті, а зовні відфільтруємо тих, хто приніс більше 500₴

query = """
SELECT *
FROM (
    SELECT user_uuid, SUM(total_uah) AS revenue
    FROM orders_log
    GROUP BY user_uuid
) AS user_revenue
WHERE revenue > 500;
"""

df = pd.read_sql(text(query), engine)
df.head()

,user_uuid,revenue
0,USR0107,4947.18
1,USR0066,19064.47
2,USR0106,19179.84
3,USR0224,18292.62
4,USR0138,18920.32


In [ ]:
query = """
SELECT user_uuid, SUM(total_uah) AS revenue
FROM orders_log
GROUP BY user_uuid
HAVING SUM(total_uah) > 500;
"""

df = pd.read_sql(text(query), engine)
df.head()

,user_uuid,revenue
0,USR0107,4947.18
1,USR0066,19064.47
2,USR0106,19179.84
3,USR0224,18292.62
4,USR0138,18920.32


### Приклад підзапиту в `WHERE`:

In [ ]:
query = """
SELECT DISTINCT user_uuid
FROM devices_users_map
WHERE user_uuid IN (
    SELECT DISTINCT user_uuid
    FROM orders_log
);
"""

df = pd.read_sql(text(query), engine)
df.head()

,user_uuid
0,USR0107
1,USR0066
2,USR0106
3,USR0224
4,USR0138


## CTE — Common Table Expression (вираз загальної таблиці)

`WITH` або CTE — це спосіб **дати ім’я підзапиту**, щоб потім використовувати його як звичайну таблицю **у зовнішньому запиті**.

### Синтаксис:

```sql
WITH ім’я_таблиці AS (
    SELECT ... -- тут підзапит
)
SELECT ...
FROM ім’я_таблиці;
```


### Приклад:

In [8]:
query = """
WITH installs AS (
    SELECT
        acquisition_channel,
        COUNT(DISTINCT device_code) AS install_count,
        SUM(cpi_uah) AS total_cpi
    FROM app_sessions
    GROUP BY acquisition_channel
),
revenue AS (
    SELECT
        app_sessions.acquisition_channel,
        SUM(orders_log.total_uah) AS total_revenue
    FROM app_sessions
    JOIN devices_users_map
        ON app_sessions.device_code = devices_users_map.device_code
    JOIN orders_log
        ON devices_users_map.user_uuid = orders_log.user_uuid
    GROUP BY app_sessions.acquisition_channel
)

SELECT
    installs.acquisition_channel,
    install_count,
    total_cpi,
    total_revenue,
    ROUND((total_revenue - total_cpi) / NULLIF(total_cpi, 0), 2) AS roi
FROM installs
LEFT JOIN revenue
    ON installs.acquisition_channel = revenue.acquisition_channel
ORDER BY roi DESC NULLS LAST;
"""

df_var_2 = pd.read_sql(text(query), engine)
df_var_2.head()

,acquisition_channel,install_count,total_cpi,total_revenue,roi
0,Organic,111,2576.55,937531.37,362.87
1,Facebook,134,3401.53,1150615.53,337.26
2,Referral,132,3369.93,994129.97,294.00
3,Google Ads,123,3196.90,913383.78,284.71


In [ ]:
df

,user_uuid
0,USR0107
1,USR0066
2,USR0106
3,USR0224
4,USR0138
...,...
295,USR0231
296,USR0008
297,USR0104
298,USR0023


In [ ]:
query = """
SELECT
acquisition_channel,
  SUM(cpi_uah)
FROM app_sessions
GROUP BY acquisition_channel
"""

df = pd.read_sql(text(query), engine)
df.head()

,acquisition_channel,sum
0,Organic,2576.55
1,Referral,3369.93
2,Facebook,3401.53
3,Google Ads,3196.90


### У чому **різниця** між CTE та підзапитом?

|                       | Вкладений підзапит (`SELECT (...)`)  | CTE (`WITH ...`)                                |
| --------------------- | ------------------------------------ | ----------------------------------------------- |
| Розташування          | Прямо в `SELECT`, `FROM`, `WHERE`    | На початку запиту                               |
| Читаємість            | Менша, особливо якщо вкладеність > 1 | Вища: структура запиту читається по кроках      |
| Повторне використання | Ні                                   | Так — можна використовувати в кількох місцях    |
| Багатоетапність       | Складно реалізувати                  | Зручно реалізовувати покрокові обчислення       |
| Продуктивність        | Зазвичай схожа                       | Може бути гіршою або кращою — залежить від СУБД |


### Коли використовувати:

| Хочеш...                                                   | Використовуй |
| ---------------------------------------------------------- | ------------ |
| Вкласти один простий `SELECT` всередину `WHERE` або `FROM` | Підзапит     |
| Розділити складне обчислення на зрозумілі кроки            | CTE          |
| Використовувати результат підзапиту в кількох місцях       | CTE          |
| Налагодити або протестувати частини логіки окремо          | CTE          |

---

> **CTE (WITH)** — це іменований підзапит, винесений наверх і зручний для повторного використання та читаємості.
> **Підзапит** — це вбудований всередину основного запиту шматок SQL-коду без імені.


# Індекси та продуктивність SQL-запитів для аналітика


## Що таке індекс

**Індекс** — це додаткова структура даних, яку СУБД (наприклад, PostgreSQL, MySQL) створює для прискореного пошуку рядків.

*Аналогія:* це як алфавітний покажчик у книзі: щоб знайти "Revenue Retention", ти не перегортаєш усю книгу, а йдеш до кінця і знаходиш сторінку за змістом.


## Як працює індекс

Коли ти робиш запит типу:

```sql
SELECT * FROM users WHERE email = 'oleg@example.com';
```

СУБД може:

* **без індексу** — просканувати всю таблицю (Full Table Scan),
* **з індексом** — знайти `email` в індексі (зазвичай дерево) і одразу перейти до потрібного рядка.

🧠 Індекс містить пари: `значення → посилання на рядок`.


## Як перевірити, чи використовується індекс (PostgreSQL)

```sql
EXPLAIN ANALYZE
SELECT * FROM users WHERE email = 'oleg@example.com';
```

Якщо використовується індекс, ти побачиш рядок на кшталт:

```
Index Scan using idx_users_email on users ...
```

Якщо ні — буде:

```
Seq Scan on users ...
```

📌 *Seq Scan* = повний перебір → повільно на великих таблицях.


## Коли індекси допомагають

| Тип запиту               | Коментар                    |
| ------------------------ | --------------------------- |
| `WHERE column = value`   | Найефективніший випадок     |
| `JOIN` по ключу          | Прискорює з’єднання         |
| `ORDER BY created_at`    | Сортування за індексом      |
| `GROUP BY channel`       | Якщо є індекс по channel    |
| `IN (...)` або `BETWEEN` | Може використовувати індекс |


## Коли індекс НЕ спрацює

| Приклад запиту                          | Чому не спрацює                         |
| --------------------------------------- | --------------------------------------- |
| `WHERE LOWER(name) = 'oleg'`            | Індекс по `name`, а не по `LOWER(name)` |
| `WHERE price * quantity > 10000`        | Обчислення, індекс не застосовується    |
| `SELECT * FROM table`                   | Немає фільтра                           |
| `WHERE created_at::date = '2023-01-01'` | Зміна типів                             |

🧠 Індекс працює лише якщо вираз у `WHERE` **збігається з тим, як створено індекс**.
Часто корисні індекси:

* по `user_id`, `order_id`, `device_code`
* по `created_at`, якщо часто фільтруєте або сортуєте за датою


## Як подивитися існуючі індекси (PostgreSQL)

```sql
SELECT
    tablename,
    indexname,
    indexdef
FROM pg_indexes
WHERE schemaname = 'public';
```

📌 Аналітик може використовувати цей запит, щоб зрозуміти: *чи існує потрібний індекс взагалі*.


## Як вибрати, на що ставити індекс

**Крок 1: Подивись часті запити**

* В `WHERE`, `JOIN`, `ORDER BY`, `GROUP BY`

**Крок 2: Виконай EXPLAIN ANALYZE** — якщо `Seq Scan` на великій таблиці → кандидат на індекс

**Крок 3: Перевір дублікати** — чи не дублюєш індекс по інших колонках


## Побічні ефекти індексів

| Мінус                               | Коментар                                 |
| ----------------------------------- | ---------------------------------------- |
| Займають місце                      | Особливо складені індекси (`col1, col2`) |
| Сповільнюють `INSERT/UPDATE/DELETE` | Кожен індекс теж потрібно оновлювати     |
| Можуть бути зайвими                 | Погіршують продуктивність                |

📌 Якщо таблиця — **log** або **events** — з мільйонами записів на день, краще не створювати зайві індекси.


## Практичний приклад: повільний запит

Запит:

```sql
SELECT COUNT(*)
FROM orders_log
WHERE user_uuid = 'abc123';
```

⛔ Працює повільно. EXPLAIN показує:

```
Seq Scan on orders_log ...
```

📈 Оптимізація:

```sql
CREATE INDEX idx_orders_user_uuid ON orders_log(user_uuid);
```

Тепер:

```
Index Scan using idx_orders_user_uuid ...
```


## Як аналітик може прискорити роботу:

* Перед важкими JOIN перевір наявність індексів по **ключах з’єднання**
* Використовуй `EXPLAIN` для частих запитів (або `auto_explain`, якщо є)
* Уникай функцій у `WHERE` без потрібних функціональних індексів
* Працюй з DBA: запропонуй створити індекс або оптимізувати існуючі


# Порядок виконання SQL-запиту та чому це важливо


## Порядок виконання SQL-запиту

SQL-запит **не виконується зверху вниз**, як його читають. СУБД інтерпретує його **за своїм логічним порядком**.

| № | Етап             | Опис                            |
| - | ---------------- | ------------------------------- |
| 1 | `FROM`           | Джерело даних, включаючи `JOIN` |
| 2 | `ON`             | Умова об’єднання при `JOIN`     |
| 3 | `WHERE`          | Фільтрація рядків               |
| 4 | `GROUP BY`       | Групування даних                |
| 5 | `HAVING`         | Фільтрація по агрегатах         |
| 6 | `SELECT`         | Вибір стовпців                  |
| 7 | `DISTINCT`       | Видалення дублікатів            |
| 8 | `ORDER BY`       | Сортування                      |
| 9 | `LIMIT / OFFSET` | Обмеження кількості рядків      |


## Чому це важливо?

### Аліаси (псевдоніми) не працюють у `WHERE`

```sql
SELECT salary + bonus AS total_income
FROM employees
WHERE total_income > 50000; -- ❌ не спрацює
```

Правильно:

```sql
SELECT salary + bonus AS total_income
FROM employees
WHERE salary + bonus > 50000;
```

`WHERE` виконується раніше за `SELECT` і не знає про `total_income`.


### `ORDER BY` використовує аліаси

Тут аліаси вже працюють:

```sql
SELECT name, salary + bonus AS total_income
FROM employees
ORDER BY total_income DESC;
```


### Фільтрація по агрегатах — тільки в `HAVING`

```sql
SELECT department, SUM(salary) AS total_salary
FROM employees
GROUP BY department
HAVING SUM(salary) > 100000;
```


### JOIN спочатку, фільтрація потім

```sql
SELECT *
FROM users
LEFT JOIN orders ON users.id = orders.user_id
WHERE orders.id IS NULL;
```

---

* Це «антиджойн»: шукаємо користувачів без замовлень
* **Антиджойн (anti-join)** — спосіб знайти рядки **з однієї таблиці**, яким **не відповідає жоден рядок** в іншій таблиці
* `LEFT JOIN` бере **всіх користувачів** з таблиці `users`
* До кожного користувача намагається «приклеїти» замовлення (`orders`), де `users.id = orders.user_id`
* Якщо замовлень немає — `orders.*` буде заповнено `NULL`
* `WHERE orders.id IS NULL` залишає **тільки рядки**, де **не знайдено відповідностей** у таблиці `orders` — тобто де `orders.id IS NULL`
* Ми **залишаємо тільки тих користувачів**, у яких **немає замовлень**, бо замовлення **не приклеїлося** під час `LEFT JOIN`

Такий підхід часто використовують, коли потрібно знайти «висячі» або «сиротині» записи — наприклад:

* Клієнти без покупок
* Товари без категорії
* Коментарі без постів і т.д.

## Підтримка аліасів у SQL-діалектах

| SQL-діалект     | Аліас у `SELECT` можна використовувати в `GROUP BY`? | Аліас у `SELECT` можна використовувати в `ORDER BY`? | Коментар                            |
| --------------- | ---------------------------------------------------- | ---------------------------------------------------- | ----------------------------------- |
| **PostgreSQL**  | ✅ Так                                                | ✅ Так                                                | Дуже гнучкий                        |
| **MySQL ≥ 8.0** | ❌ Ні                                                 | ✅ Так                                                | В `GROUP BY` потрібно писати вираз  |
| **MySQL < 8.0** | ❌ Ні                                                 | ✅ Так                                                | Старі версії                        |
| **SQLite**      | ❌ Ні                                                 | ✅ Так                                                | Тільки `ORDER BY` бачить аліас      |
| **SQL Server**  | ❌ Ні                                                 | ✅ Так                                                | `GROUP BY` вимагає початковий вираз |
| **Oracle**      | ❌ Ні                                                 | ✅ Так                                                | `GROUP BY` не підтримує аліаси      |
| **BigQuery**    | ✅ Так                                                | ✅ Так                                                | Повна підтримка                     |


# Об’єднання даних з різних джерел


## **Parquet файл (.parquet)**

**Parquet** — це *колонковий* формат зберігання даних, який використовується для ефективної роботи з великими таблицями. Особливо популярний у аналізі даних, машинному навчанні та сховищах даних.

## В чому відмінність від CSV?

| Характеристика        | CSV                             | Parquet                                   |
| --------------------- | ------------------------------- | ----------------------------------------- |
| Формат                | Текстовий                       | Бінарний, колонковий                      |
| Розмір файлу          | Великий                         | Менше (стиснення у 2–10 разів)            |
| Швидкість читання     | Повільніше                      | Швидше при вибірці по стовпцях            |
| Часткове читання      | Ні                              | Так (можна вибрати лише потрібні стовпці) |
| Підтримка типів даних | Ні (усі рядки як текст)         | Так (int, float, datetime тощо)           |
| Зручність для очей    | Так (можна відкрити в блокноті) | Ні (бінарний формат)                      |

---

## Чому "колонковий"?

CSV та Excel — **рядкові** (row-oriented): дані читаються рядок за рядком.

Parquet — **колонковий** (column-oriented): дані читаються по стовпцях. Це дає переваги:

* Можна читати лише потрібні стовпці (економія I/O)
* Краще стиснення (однорідні дані в стовпці)
* Швидші агрегації та фільтрації по колонці

---

## Де використовується Parquet?

| Сфера                | Приклад використання                                  |
| -------------------- | ----------------------------------------------------- |
| Хмарні сховища даних | Amazon S3, Google Cloud Storage + BigQuery            |
| Аналітика / ETL      | Apache Spark, Dask, Presto, Pandas                    |
| Машинне навчання     | Підготовка ознак (features) для моделей               |
| Сховища даних (DWH)  | Parquet — стандарт де-факто для lakehouse-архітектури |

---

## Приклад: коли використовувати Parquet

Ти експортуєш 10 млн рядків про замовлення клієнтів.
Якщо це CSV — вага файлу може бути 1 ГБ.
У форматі Parquet — 100–300 МБ.
Якщо потрібно обчислити середній чек — можна прочитати **тільки стовпець з сумою покупки**.

---

## Як зберегти і прочитати в Pandas:

```python
df.to_parquet("data.parquet")        # зберегти
df = pd.read_parquet("data.parquet") # прочитати
```

> Для роботи з `.parquet` потрібні бібліотеки: `pyarrow` або `fastparquet`.

Встановити можна так:

```bash
pip install pyarrow
```


## Приклад

In [ ]:
# Get Data from DB
query = """
SELECT *
FROM orders_log
"""
orders = pd.read_sql(text(query), engine)
orders.head()

,user_uuid,order_time,total_uah
0,USR0060,2024-03-08,683.38
1,USR0060,2024-01-27,2222.03
2,USR0060,2024-03-16,2924.31
3,USR0060,2024-06-01,2914.72
4,USR0060,2024-05-14,2313.61


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving users_extended.parquet to users_extended.parquet


In [ ]:
# Get Data from Parquet
users = pd.read_parquet("users_extended.parquet")
users.head()

,user_uuid,gender,age,segment,city
0,USR0190,M,58,vip,Odesa
1,USR0038,M,24,regular,Kharkiv
2,USR0148,F,21,regular,Kharkiv
3,USR0006,M,62,newcomer,Dnipro
4,USR0026,F,28,regular,Dnipro


In [ ]:
print(orders.shape)
print(users.shape)

(2559, 3)
(300, 5)


In [ ]:
# INNER JOIN (тільки збіги)
df = pd.merge(orders, users, on='user_uuid', how='inner')
df.head()

,user_uuid,order_time,total_uah,gender,age,segment,city
0,USR0213,2024-04-22,2714.80,F,38,regular,Dnipro
1,USR0213,2024-03-22,2430.78,F,38,regular,Dnipro
2,USR0213,2024-06-03,2263.12,F,38,regular,Dnipro
3,USR0213,2024-01-06,330.05,F,38,regular,Dnipro
4,USR0213,2024-06-11,2249.34,F,38,regular,Dnipro


In [ ]:
df.shape

(2559, 7)

In [ ]:
# LEFT JOIN (всі замовлення, навіть без інформації про користувача)
df = pd.merge(orders, users, on='user_uuid', how='left')
df.head()

,user_uuid,order_time,total_uah,gender,age,segment,city
0,USR0060,2024-03-08,683.38,NaN,NaN,NaN,NaN
1,USR0060,2024-01-27,2222.03,NaN,NaN,NaN,NaN
2,USR0060,2024-03-16,2924.31,NaN,NaN,NaN,NaN
3,USR0060,2024-06-01,2914.72,NaN,NaN,NaN,NaN
4,USR0060,2024-05-14,2313.61,NaN,NaN,NaN,NaN


In [ ]:
# RIGHT JOIN (всі користувачі, навіть без замовлень)
df = pd.merge(orders, users, on='user_uuid', how='right')
df.head()

,user_uuid,order_time,total_uah,gender,age,segment,city
0,USR0190,2024-05-19,591.17,M,58,vip,Odesa
1,USR0190,2024-01-28,2924.27,M,58,vip,Odesa
2,USR0038,2024-01-19,2976.64,M,24,regular,Kharkiv
3,USR0038,2024-01-18,2428.07,M,24,regular,Kharkiv
4,USR0038,2024-05-01,2434.18,M,24,regular,Kharkiv


In [ ]:
df.shape

(915, 7)

In [ ]:
# FULL OUTER JOIN (всі дані: і замовлення, і користувачі)
df = pd.merge(orders, users, on='user_uuid', how='outer')
df.head()

,user_uuid,order_time,total_uah,gender,age,segment,city
0,0029e34f-6d0b-41fb-9501-79b757f157b9,NaN,NaN,F,34.0,regular,Lviv
1,006ecdca-5c8d-4a6c-874e-69abda1dfd3b,NaN,NaN,F,42.0,newcomer,Kyiv
2,01cede6c-4e61-4564-9437-8e450d5f2539,NaN,NaN,F,54.0,regular,Lviv
3,0208aa4c-32de-4364-aab9-38daaeeec547,NaN,NaN,M,38.0,regular,Lviv
4,028934c6-588f-4ee3-857a-37d69b8215d2,NaN,NaN,M,38.0,regular,Kyiv


In [ ]:
df.shape

(2769, 7)